In [1]:
from whstudy import WorldIndicators
from sklearn.manifold import TSNE
from ProjectionCorrelation import score_projections
from Orange.data.pandas_compat import table_from_frame
from Orange.data import ContinuousVariable

import random

### Get data from mongo database

In [2]:
handle = WorldIndicators('main', 'biolab')
indicator_codes = [code for code, _, _, _ in handle.indicators()]
country_codes = [code for code, _ in handle.countries()]
random.seed(0)
indicator_codes = random.sample(indicator_codes, 20)
country_codes = random.sample(country_codes, 20)

df = handle.data(country_codes, indicator_codes, 2008)

### Split data to reference and overlay and perform t-SNE

In [3]:
reference = df.iloc[:,:10].fillna(0)
overlay = df.iloc[:,10:].fillna(0)

ref_tsne = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(reference.to_numpy())

tSNE_X = ContinuousVariable("t-SNE-x")
tSNE_Y = ContinuousVariable("t-SNE-y")

overlay_table = table_from_frame(overlay)
ref_table = table_from_frame(reference)
ref_table = ref_table.add_column(tSNE_X, ref_tsne[:, 0], to_metas=True)
ref_table = ref_table.add_column(tSNE_Y, ref_tsne[:, 1], to_metas=True)

scores = score_projections(ref_table, tSNE_X, tSNE_Y, overlay_table)

TypeError: not all arguments converted during string formatting